# import libraries

In [7]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes

In [2]:
from compas.geometry import Point, Frame, Box
from compas_vol.primitives import VolBox
from compas_vol.microstructures import TPMS
from compas_vol.modifications import Overlay

# create volumetric objects

In [25]:
box = Box(Frame(Point(0,0,0), (1,0.2,-0.3), (0,1,0.4)), 25, 20, 15)
vbox = VolBox(box, 4.0)
gyr = TPMS(0,1.5)
ov = Overlay(vbox, gyr, 0.4)

## sample at discrete intervall (dense grid)

In [26]:
x, y, z = np.ogrid[-20:20:128j,-20:20:128j,-20:20:128j]
dm = ov.get_distance_numpy(x, y, z)

In [27]:
v, f, n, l = marching_cubes(dm, 0)

In [28]:
p = mp.plot(v,f, shading={"flat":False, "width":800, "height":450})

Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, children=(DirectionalLight(color='white', intensi…

In [30]:
p = mp.plot(v,f, shading={"flat":False, "width":800, "height":450})

@mp.interact(fc=(1000,50000))
def facefilter(fc=3000):
    p.update_object(faces=f[:fc])

Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, children=(DirectionalLight(color='white', intensi…

interactive(children=(IntSlider(value=3000, description='fc', max=50000, min=1000), Output()), _dom_classes=('…

## visualize isosurface

In [15]:
import ipyvolume as ipv
fig = ipv.figure(width=800, height=450)
ipv.xyzlim(20)
mesh = ipv.plot_isosurface(dm, 0.0, color='white', extent=[(-20,20) for _ in range(3)])
ipv.style.use('minimal')
ipv.show()

/Users/bernham/anaconda3/envs/compasdev/lib/python3.9/site-packages/ipyvolume/pylab.py:742: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level)


# apply low pass filter using fourier transform

In [16]:
dft = np.fft.fftn(dm)
dfts = np.fft.fftshift(dft)
lowpass = np.zeros_like(dft,dtype=np.complex128)
fw = 12
a = (128-fw)//2
b = a+fw
lowpass[a:b,a:b,a:b] = dfts[a:b,a:b,a:b]
lowpass = np.fft.fftshift(lowpass)
idft = np.fft.ifftn(lowpass)

## visualize smoothed version

In [17]:
v, f, n, l = marching_cubes(idft.real, 0)
p = mp.plot(v,f, shading={"flat":False, "width":800, "height":450})

Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, children=(DirectionalLight(color='white', intensi…

In [28]:
fig = ipv.figure(width=800, height=450)
ipv.xyzlim(20)
mesh = ipv.plot_isosurface(idft.real, 0.0, color='pink', extent=[(-20,20) for _ in range(3)])
ipv.style.use('minimal')
ipv.show()

/Users/bernham/anaconda3/envs/compasdev/lib/python3.9/site-packages/ipyvolume/pylab.py:742: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level)
